# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
# ! mongoimport --type json -d uk_foods -c establishments --drop --jsonArray establishments.json

In [2]:
# Import dependencies
from pymongo import MongoClient
import json
import requests
from pprint import pprint
import pandas as pd
import collections


In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'claddDB', 'classDB', 'config', 'fruit_db', 'local', 'met', 'petsitly_marketing', 'uk_foods']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_foods']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# assign the collection to a variable
establishments = db['establishments']
establishments

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_foods'), 'establishments')

In [8]:
# review the collections in our new database
establishment_documents = establishments.count_documents({})
establishment_documents

38793

In [9]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6410c641053464abc3d56ae9'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [10]:
# Create a dictionary for the new restaurant data
item_to_add = []

item_to_add = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [11]:
# Insert the new restaurant into the collection

establishments.insert_one(item_to_add)
    

In [12]:
# Check that the new restaurant was inserted

print(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'_id': ObjectId('6410dc30912fede6b6b04a73'), 'BusinessName': 'Penang Flavours', 'AddressLine1': 'Greenwich', 'BusinessTypeID': '1'}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [13]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields

query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
         
fields = {"BusinessTypeID": 1, "BusinessType": 1}
        
results = list(establishments.find(query, fields))

pprint(results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 '_id': ObjectId('6410c641053464abc3d56aeb')}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [14]:
# Update the new restaurant with the correct BusinessTypeID

establishments.update_one({"BusinessName": "Penang Flavours"}, {"$set": {"BusinessTypeID": "1"}})

In [15]:
# Confirm that the new restaurant was updated
print(establishments.find_one({"BusinessName":"Penang Flavours"}))

{'_id': ObjectId('6410dc30912fede6b6b04a73'), 'BusinessName': 'Penang Flavours', 'AddressLine1': 'Greenwich', 'BusinessTypeID': '1'}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [16]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({"LocalAuthorityName": "Dover"})

0

In [17]:
# Delete all documents where LocalAuthorityName is "Dover"

establishments.delete_many({"LocalAuthorityName": "Dover"})


In [18]:
# Check if any remaining documents include Dover
pprint(establishments.count_documents({"LocalAuthorityName": "Dover"}))

0


In [19]:
# Check that other documents remain with 'find_one'
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6410c641053464abc3d56ae9'),
 'geocode': {'latitude': '51.086058', 'longitude': '1.196408'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          '

5. Some of the number values are stored as strings, when they should be stored as numbers. Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [20]:
# Change the data type from String to Decimal for longitude and Latitude

establishments.update_many({}, [{'$set': {"geocode.latitude": {'$toDecimal': '$geocode.latitude'}, "geocode.longitude": {'$toDecimal': '$geocode.longitude'}}}])
pprint(db.establishments.find_one())

{'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'ChangesByServerID': 0,
 'Distance': 4591.821311183521,
 'FHRSID': 647177,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'LocalAuthorityCode': '188',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT19 6PY',
 'RatingDate': '2014-03-31T00:00:00',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingValue': '4',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6410c641053464abc3d56ae9'),
 'geocode': {'latitude': Decimal128('51.086058'),
             'longitude': Decimal128('1.196408')},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/647177',
            'rel': 'self'}],
 'me